In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

In [2]:
url = "https://redplanetscience.com/"

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been s

In [4]:
# NASA Mars News Scrape
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [5]:
latest_article = soup.select_one("div.list_text")

In [6]:
news_title = latest_article.find('div', class_='content_title').text
news_desc = latest_article.find('div', class_='article_teaser_body').text
print(news_title)
print(news_desc)

NASA's Treasure Map for Water Ice on Mars
A new study identifies frozen water just below the Martian surface, where astronauts could easily dig it up.


In [7]:
# Mars Featured Image Scrape
mars_images_url = "https://spaceimages-mars.com/"
browser.visit(mars_images_url)
html = browser.html
soup = bs(html, 'html.parser')

In [8]:
results = soup.find('img', class_='headerimage')
print(results.prettify())

<img class="headerimage fade-in" src="image/featured/mars3.jpg"/>



In [28]:
source = results['src']
featured_image_url = f'https://spaceimages-mars.com/{source}'
featured_image_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [12]:
# Mars Facts Scrape
mars_facts_url = "https://galaxyfacts-mars.com/"

In [13]:
tables = pd.read_html(mars_facts_url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [22]:
facts_df = pd.DataFrame(tables[0])
facts_df.rename(columns={0: 'Mars - Earth Comparison', 
                         1: 'Mars', 
                         2: 'Earth'}, inplace=True)
facts_df

,Mars - Earth Comparison,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [23]:
facts_df.to_html('mars_facts_table.html')

In [49]:
# Mars Hemispheres Scrape
browser.visit("https://marshemispheres.com/")
html = browser.html
soup = bs(html, 'html.parser')
products = soup.body.find_all('div', class_='description')

In [50]:
click_links = []
products = soup.body.find_all('div', class_='description')
for product in products:
    link_name = product.h3.text
    click_links.append(link_name)
click_links

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [55]:
hemisphere_image_urls = []

for x in range(0, len(click_links)):
    browser.links.find_by_partial_text(click_links[x]).click()
    
    html = browser.html
    soup = bs(html, 'html.parser')
    
    image = soup.find('img', class_='wide-image')['src']
    hemisphere = click_links[x][:-9]
    
    print('----------')
    print(f'{hemisphere}')
    print(f'https://marshemispheres.com/{image}')
    
    hemisphere_dict = {}
    hemisphere_dict['title'] = hemisphere
    hemisphere_dict['img_url'] = f'https://marshemispheres.com/{image}'
    hemisphere_image_urls.append(hemisphere_dict)
    
    browser.visit("https://marshemispheres.com/")

----------
Cerberus Hemisphere
https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
----------
Schiaparelli Hemisphere
https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
----------
Syrtis Major Hemisphere
https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
----------
Valles Marineris Hemisphere
https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg


In [56]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [57]:
browser.quit()